# <center> Topic Modeling </center>

- This lecture is created based on 
    * Blei. D. (2012), "Probabilistic Topic Models", http://www.cs.columbia.edu/~blei/talks/Blei_ICML_2012.pdf
    * Topic Modeling with Scikit Learn: https://medium.com/@aneesha/topic-modeling-with-scikit-learn-e80d33668730

## 1. Introduction

- Topic modeling provides methods for automatically organizing, understanding, searching, and summarizing large electronic archives.
  - Discover the hidden themes that pervade the collection.
  - Annotate the documents according to those themes.
  - Use annotations to organize, summarize, and search the texts.
- Formal Definition
<img src='topic_modeling.png' width='70%'>
  - **Topics**: each topic is a **distribution over words** 
    - e.g. for topic "Gentics", $p('gene'~|~'Genetics')~=~0.04$, $p('dna'~|~'Genetics')=0.02$
    - $K$ topics $\theta_1, \theta_2, ..., \theta_K$, $N$ words $w_1, w_2, ..., w_N$ in corpus, we need to know  $p(w_i|\theta_j)$ for $i \in N$ and $j\in K$
  - **Document ($d$)**: a **mixture of topics**
    - e.g. for above document $d$, $p('Genetics'~|~d)=0.5$, $p('LifeScience'~|~d)=0.15$, ...
    - In general, given document $d$ and topic $\theta_j$, we need to know $p(\theta_j~|~d)$, i.e. **topic proportion**


## 2. Statistical Language Model
- Definition: given a corpus with $M$ documents, $N$ words, $K$ topics, a model contains the following probabilities:
  - topic probability distribution in corpus: <br>$p(\theta_j)$ for $j \in K$, $\sum_{j\in K}{p(\theta_j)}=1$
  - topic distribution per document $d$ (document assignment): <br>$p(\theta_j~|~d)$, $\sum_{j\in K}{p(\theta_j~|~d)}=1$ 
  - word distribution per topic (why do we need to know it?): <br> $p(w_i~|~\theta_j)$ for $i \in N$ and $j\in K$, $\sum_{i\in N}{p(w_i~|~\theta_j)}=1$ 
  <img src='language_model.png' width='30%'>

## 3. How to estimate these probabilities?
### 3.1. Supervised learning - Naive Bayes
- Topic probability: 
$$ p(\theta_j) = \frac{\text{documents in topic } j} {\text{total documents}}$$  
- Word distribution per topic: 
$$ p(w_i~|~\theta_j)= \frac{\text{count of word } w_i \text{ in topic } j} {\text{total word count in documents of topic }j}$$  
- Topic distribution per document: 
$$ \begin{array}{l}
 p(\theta_j~|~d) = \frac{p(d|\theta_j) * p(\theta_j)}{p(d)} \text{         # Bayesian rule}\\
 C_{MAP} = \underset{\theta}{\operatorname{argmax}}{p(d~|~\theta)*p(\theta)} \text{         # maximum a posteriori}\\
    C_{MAP} = \underset{\theta}{\operatorname{argmax}}{p(w_1,w_2, ...,w_N~|~\theta)*p(\theta)} \\
    C_{MAP} = \underset{\theta}{\operatorname{argmax}}({\prod_{i \in N} {p(w_i~|~\theta)})*p(\theta)}  \textit{ # independence assumption}
  \end{array}$$  
- Naive Bayes model is also a kind of language model

### 3.2. Generative Model for Unsupervised learning 
- We don't have labeled data; we only observe the documents
- We **cannot** estimate $p(\theta_j)$ and $p(w_i~|~\theta_j)$ as above
- Instead, we use a **generative model** that describes how a document $d$ was created
  1. decide on document length $N$, e.g. 100 words
  2. decide on topic mixture (i.e. $p(\theta_j~|~d)$), e.g. 70% about genetics and 30% about life science, ...
  3. for each of the N words,
     - 3.1. choose a topic from the topic mixture, e.g. "genetics"
     - 3.2. choose a word from based on the probabilities of words in the topic (i.e. $p(w_i~|~\theta_j)$), e.g. "gene"
     - At the end, you may get a document such as "gene dna life ..."
- We assume all documents in the dataset were generated following this process. Then we infer these probabilities from samples such that these probabilities have the maximum likelihood to generate the samples
- Probabilities $p(w_i~|~\theta_j)$ and $p(\theta_j~|~d)$ are **hidden structures** to be discovered, a.k.a **latent variables**
<img src='latent_structure.png' width='70%'>
  


## 4. Latent Dirichlet Allocation (LDA)
- A generative model which generates a document $d$ as follows:
  1. Choose document length $N$ ∼ Poisson(ξ).
  2. Choose topic mixture $\theta$ ~ Dir(α).
  3. For each of the $N$ words $w_n$:
     - (a) Choose a topic assignment $z_n$ ∼ Multinomial(θ).
     - (b) Choose a word $w_n$ from the topic, $z_n$ ∼ Multinomial($\beta_{z_n}$), where $\beta_{z_n}$ is the word distribution for assigned topic $z_n$, i.e. $p(w_n~|~z_n)$     
- A few distributions
  - Poisson(ξ) : a given number of events occurring in a fixed interval of time/space with rate ξ independently of the time/space since the last event
  - Multinomial(θ) & Multinomial($\beta$): 
    - suppose X is a vector which represents n draws of a random variable with three possible outcomes (i.e. words), say A, B, C. 
    - e.g. when n=10, an example draw of X could be x = [4,4,2], i.e., A occured 4 times, B 4 times, and C 2 times 
    - assume three outcomes have probability θ={$\beta_A$, $\beta_B$,$\beta_C$} respectively (i.e. 0.5,0.3,0.2)
    - the multinomial distribution describes the prob. mass distribution of X, $$ multinomial(X=[4,4,2]) = \frac{10!}{4!4!2!}\beta_A^{4}\beta_B^{4}\beta_C^{2}$$ 
  - Dir(α) : is a probability distribution with parameter $α, e.g. \{α_1,α_2,α_3\}$ to generate $θ, e.g. \{ θ_1,θ_2,θ_3\}$. For details of Dirichlet function, check videos e.g. https://www.youtube.com/watch?v=nfBNOWv1pgE 
  <img src='dirichlet.svg'>
    - Dirichlet distribution is conjugate to the multinomial.
    - Given a multinomial observation, the posterior distribution of θ is a Dirichlet.
    - In LDA, usually $α_1=α_2=α_3=...=\frac{1}{K}$
- Common techniques to estimate these probabilities are EM (Expectation-Maximization), Gibbs Sampling (See Blei's paper for details)


### 4.1. Estimate parameters by Gibbs Sampling
- General ideas of Gibbs sampling
  - In statistics, Gibbs sampling or a Gibbs sampler is a Markov chain Monte Carlo (MCMC) algorithm for obtaining a sequence of observations which are approximated from a specified multivariate probability distribution, when direct sampling is difficult (from *Wikipedia*).
  - e.g.  by bayes inference, $p(x, y)=p(x|y)~p(y)=p(y|x)~p(x)$. If it's difficult to determine the nature of $p(x, y)$, while it is easy to sample from $p(y|x)$  and $p(x|y)$, then we can obtain a sequence of observations approximating $p(x, y)$ by sampling $p(y|x)$ and $p(x|y)$. 
  - Let's assume:
    * $p(y|x)$ \~ $N(x,\sigma_1)$
    * $p(x|y)$ \~ $N(y,\sigma_2)$ 
    * Now we need to have 100 $(x,y)$ samples so that we can understand the nature of $p(x,y)$
  - The Gibbs sampler proceeds as follows:
    1. set $x$ and $y$ to some initial starting values 
    2. Replace $x$ by a new value obtained by sampling $x|y$, then update $y$ by sampling from $y|x$, 
    3. Repeat step 2 until the estimates of parameters converge.
- Outline of Gibbs Sampling in LDA
  1. Go through each document and randomly assign each word in the document to one of the $K$ topics, i.e. $p(z_i=j~|~w_i, d)$, where $z_i=j$ denotes word $i$ is assigned to topic $j$. 
  2. Calculate the following
     - $p(w_i~|~\theta_j)$ (word-topic matrix): calculated as the count of each word being assigned to each topic. 
     - $p(\theta_j~|~d)$ (document-topic matrix): the number of words assigned to each topic for each document 
  3. Update $p(z_i=j~|~w_i, d)$ using $p(w_i~|~\theta_j)$ and $p(\theta_j~|~d)$ as follows:
     - For each document $d$, and each word $w_i$, reassign a new topic $j$ to $w_i$, where we choose topic $j$ is sampled from [1, 2, ..., $K$] with a probability ${\propto}~ (w_i~|~\theta_j) * p(\theta_j~|~d)$ (conditional posterior distribution)
     - i.e. Given $w_i$, $d$, $z_{-i}$ (topic assignment of all other words in $d$), $$p(z_i=j~|~w_i, z_{-i}, d)~{\propto}~p(w_i~|~z_i=j, z_{-i}, d) * p(z_i=j~|~z_{-i}, d)~{\propto}~(w_i~|~\theta_j) * p(\theta_j~|~d)$$ 
  4. Repeat steps 2-3 until $p(w_i~|~\theta_j)$ and  $p(\theta_j~|~d)$ converge.
- For more implementation details, check http://ethen8181.github.io/machine-learning/clustering_old/topic_model/LDA.html

### 4.2 Evaluate Topic Model - Perplexity
- For a single document $d$ with $N_d$ words $\{w_1, w_2, ..., w_{N_d}\}$, denoted as $\textbf{W}_d$
$$
  perplexity(d)= exp({H(d)}),  
  H(d) = - \frac{ln (p(\textbf{W}_d))}{N_d}  
$$
- $p(\textbf{W}_d)$, the probability of seeing a document $d$, can be calculated based on:
   - word distribution per topic, i.e. $p(w_i~|~\theta_j$, and 
   - topic mixture, i.e. $p(\theta_j~|~d)$
- For a test set of D with M documents
$$ perplexity(d)= exp({H(D)}), H(D) = - \frac{\sum_{d \in D} {ln   (p(\textbf{W}_d)})}{\sum_{d \in D}{N_d}} $$
- Intutition: 
  - A lower perplexity score indicates better generalization performance
  - Minimizing H(d) is equivalent to maximizing log likelihood
- To evaluate a topic model, calcuate perplexity on **testing dataset** (i.e. evaluating how generaalized the model is)
- Note: if you have some labeled data, you should also conduct **external evaluation**, i.e. 
  - map each topic to a labeled class, 
  - compute precision/recall from the labeled data

## 5. Experiement with LDA
- A few libraries available for LDA: gensim, lda, sklearn
- We use sklearn here since it has a good text preprocessing module

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Exercise 5.1. Load 20 news group data
import json
from numpy.random import shuffle

data=json.load(open('ydata.json','r'))

# shuffle the data
shuffle(data)

text,label=zip(*data)
text=list(text)
label=list(label)

print(text[0])
print(label[0])

christy bareijsza the event guru for weddings and high profile events christy bareijsza president of the red carpet events known to many as the event guru is the expert of experts in the field of event planning and weddings continues to produce stellar events and provide memorable experiences in new york ny vocus february christy bareijsza president of the red carpet events known to many as the event guru is the expert of experts in the field of event planning and weddings continues to produce stellar events and provide memorable experiences in christy bareijsza the event guru and a go to source for creatively produced events from soirees to corporate events to one of a kind special weddings that are different from the norm continue to expand business this year with her many years in the business christy really personalizes events and exceeds client expectations by taking it to another level and achieving the best results for them the red carpet events is a national full service planni

In [4]:
# Exercise 5.2. Preprocessing - Create Term Frequency Matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# LDA can only use raw term counts for LDA 
tf_vectorizer = CountVectorizer(max_df=0.90, \
                min_df=50, stop_words='english')
tf = tf_vectorizer.fit_transform(text)

# each feature is a word (bag of words)
# get_feature_names() gives all words
tf_feature_names = tf_vectorizer.get_feature_names()

print(tf_feature_names[0:10])
print(tf.shape)

# split dataset into train (90%) and test sets (10%)
# the test sets will be used to evaluate proplexity of topic modeling
X_train, X_test = train_test_split(\
                tf, test_size=0.1, random_state=0)

['aaron', 'abandoned', 'abc', 'ability', 'able', 'abroad', 'absolutely', 'abuse', 'academy', 'accelerate']
(6426, 4273)


In [1]:
# Exercise 5.3. Train LDA model
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 4

# Run LDA. For details, check
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation.perplexity

# max_iter control the number of iterations 
# evaluate_every determines how often the perplexity is calculated
# n_jobs is the number of parallel threads
lda = LatentDirichletAllocation(n_components=num_topics, \
                                max_iter=10,verbose=1,
                                evaluate_every=1, n_jobs=1,
                                random_state=0).fit(X_train)

NameError: name 'X_train' is not defined

In [6]:
# Exercise 5.4. Check topic and word distribution per topic

num_top_words=20

# lda.components_ returns a KxN matrix
# for word distribution in each topic.
# Each row consists of 
# probability (counts) of each word in the feature space

for topic_idx, topic in enumerate(lda.components_):
    print ("Topic %d:" % (topic_idx))
    # print out top 20 words per topic 
    words=[(tf_feature_names[i],topic[i]) for i in topic.argsort()[::-1][0:num_top_words]]
    print(words)
    print("\n")

Topic 0:
[('company', 4097.7329422535395), ('million', 3646.8402730069529), ('year', 3288.3787458435909), ('percent', 2876.7522469879341), ('market', 2788.796747882815), ('income', 2601.2254544436028), ('quarter', 2442.1550698475585), ('financial', 2150.8632480496358), ('investment', 1972.7158580685209), ('share', 1868.8986225478534), ('billion', 1858.4448347071636), ('net', 1848.2449786148386), ('shares', 1752.020563130174), ('statements', 1750.806256200445), ('sales', 1656.5528101873608), ('fund', 1641.0306277938391), ('investors', 1613.3812554948117), ('term', 1610.8207360051265), ('stock', 1586.6853991484536), ('cash', 1446.5841525969404)]


Topic 1:
[('com', 5408.1223276165229), ('new', 3558.1679675525525), ('company', 3260.2581225149183), ('www', 3096.7643615433763), ('information', 3064.9179944050807), ('business', 3042.8157692704162), ('services', 2306.0752692873534), ('technology', 1934.0792089888478), ('products', 1922.8351509529091), ('http', 1703.975928210135), ('service', 

In [7]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math

num_top_words=50
f, axarr = plt.subplots(2, 2, figsize=(8, 8));

for topic_idx, topic in enumerate(lda.components_):
    # create a dataframe with two columns (word, weight) for each topic
    
    # create a word:count dictionary
    f={tf_feature_names[i]:topic[i] for i in topic.argsort()[::-1][0:num_top_words]}
    
    # generate wordcloud in subplots
    wordcloud = WordCloud(width=480, height=450, margin=0, background_color="black");
    _ = wordcloud.generate_from_frequencies(frequencies=f);
    
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].imshow(wordcloud, interpolation="bilinear");
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].set_title("Topic: "+str(topic_idx));
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].axis('off')

plt.tight_layout()
plt.show()



ModuleNotFoundError: No module named 'wordcloud'

In [8]:
# Exercise 5.5. Assign documents to topic
import numpy as np

# Generate topic assignment of each document
topic_assign=lda.transform(X_train)

print(topic_assign[0:5])

# set a probability threshold
# the threshold determines precision/recall
prob_threshold=0.25

topics=np.copy(topic_assign)
topics=np.where(topics>=prob_threshold, 1, 0)
print(topics[0:5])


[[  8.87837194e-01   7.73974678e-04   1.10598931e-01   7.89900328e-04]
 [  2.00541510e-03   1.53537902e-01   1.90334916e-03   8.42553334e-01]
 [  9.86093589e-02   8.90852147e-01   9.63476458e-03   9.03729710e-04]
 [  5.26863526e-03   5.32221412e-03   5.24349697e-03   9.84165654e-01]
 [  3.72123102e-01   5.73472824e-01   5.34907281e-02   9.13346123e-04]]
[[1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [1 1 0 0]]


In [9]:
# Exercise 5.6. Evaluate topic models by perplexity of test data

perplexity=lda.perplexity(X_test)
print(perplexity)


1991.85103494


### 5.1. Find the number of topics ($K$)
- There are no "golden" rules to find K.
- Perplexity may be one way for you to find the number of topics
    - Typically, the best number of topics should be around the **lowest perplexity**
- However, in practice, a few factors need to be considered:
  - It is usually difficult for human to understand or visulaize a big number of topics (I'd suggest K < 20)
  - You may manually scan the data to figure out possible topics in the data, but these topics may not be correlated with the hidden structure discovered by LDA
  - Usually, after LDA, we need manually inspect each discovered topic, merge or trim topics to get semantically coherent but distinguishable topics

In [ ]:
# Exercise 5.7. How to find the best number of topics?
# Vary variable num_topics, e.g. set it to 2, 3, 5, ...
# For each value, train LDA model, calculate perplexity on the test data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

result=[]
for num_topics in range(2,25):
    lda = LatentDirichletAllocation(n_components=num_topics, \
                                learning_method='online', \
                                max_iter=10,verbose=0, n_jobs=1,
                                random_state=0).fit(X_train)
    p=lda.perplexity(X_test)
    result.append([num_topics,p])
    print(num_topics, p)



In [ ]:
import pandas as pd
pd.DataFrame(result, columns=["K", "Perlexity"]).plot.line(x='K',y="Perlexity");
plt.show();

## 6. Non-Negative Matrix Factorization (NMF)

- NMF is similar to LDA with different mathematical underpinning. It decompose document-term matrix into the product of 
  - feature matrix (i.e. word distribution per topic) and 
  - weight matrix (i.e. topic mixture per document) 
- NMF is very efficient for small matrix

In [10]:
# Exercise 6.1. NMF transformation

from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=20, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(text)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print(tfidf_feature_names[0:10])
print(tfidf.shape)

X_train, X_test = train_test_split(tfidf, test_size=0.1, random_state=0)

no_topics = 4

# Run NMF
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html
# init: ‘nndsvd’: Nonnegative Double Singular Value Decomposition (NNDSVD) better for sparseness
# alpha: regularization

nmf = NMF(n_components=no_topics, \
          random_state=1, alpha=0.01, init='nndsvd').fit(X_train)



['aa', 'aaa', 'aapl', 'aaron', 'abandoned', 'abandons', 'abc', 'ability', 'able', 'aboard']
(6426, 7805)


In [11]:
# Exercise 6.2. Get topic words

num_top_words=20

# lda.components_ returns a KxN matrix
# for word distribution in each topic.
# Each row consists of 
# probability (counts) of each word in the feature space

for topic_idx, topic in enumerate(nmf.components_):
    print ("Topic %d:" % (topic_idx))
    # print out top 20 words per topic 
    words=[(tfidf_feature_names[i],topic[i]) \
           for i in topic.argsort()[::-1][0:num_top_words]]
    print(words)
    print("\n")

Topic 0:
[('company', 0.74036508210084306), ('com', 0.63777863023414172), ('percent', 0.62984743226844031), ('million', 0.46593381109382698), ('business', 0.42974376451146079), ('quarter', 0.42073227747783232), ('sales', 0.40826369894643905), ('market', 0.39767646660976869), ('www', 0.39126724986127481), ('new', 0.37201271539444963), ('products', 0.35614132184657016), ('companies', 0.34451163490097247), ('statements', 0.34184716339966126), ('information', 0.32066671931600393), ('year', 0.31944377175250938), ('stock', 0.31750365325711311), ('billion', 0.31356519403250505), ('shares', 0.30937166806306488), ('services', 0.30747885805925862), ('technology', 0.30111970735989957)]


Topic 1:
[('said', 1.0995160128244907), ('police', 0.86509937373860779), ('court', 0.406698709998705), ('man', 0.38034306820055769), ('comment', 0.3627650065908789), ('year', 0.31221340823666105), ('old', 0.30529146170312871), ('told', 0.30255268378416278), ('people', 0.2893640273118217), ('say', 0.27185408246071

In [12]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math

num_top_words=50
f, axarr = plt.subplots(2, 2, figsize=(8, 8));

for topic_idx, topic in enumerate(nmf.components_):
    # create a dataframe with two columns (word, weight) for each topic
    
    # create a word:count dictionary
    f={tfidf_feature_names[i]:topic[i] \
       for i in topic.argsort()[::-1][0:num_top_words]}
    
    # generate wordcloud in subplots
    wordcloud = WordCloud(width=480, height=450, margin=0, background_color="black");
    _ = wordcloud.generate_from_frequencies(frequencies=f);
    
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].imshow(wordcloud, interpolation="bilinear");
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].set_title("Topic: "+str(topic_idx));
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].axis('off')

plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'wordcloud'

In [13]:
# Exercise 6.3. Assign document to topics

import numpy as np
from sklearn.preprocessing import normalize

topic_assign=nmf.transform(X_train)

print(topic_assign[0:5])

topics=normalize(np.copy(topic_assign), axis=1, norm='l1')
print(topics[0:5])

prob_threshold=0.25

topics=np.where(topics>prob_threshold, 1, 0)
print(topics[0:5])

[[ 0.03209943  0.0010115   0.01927236  0.04786201]
 [ 0.01096943  0.04006777  0.          0.        ]
 [ 0.05673413  0.00226616  0.          0.        ]
 [ 0.          0.04880261  0.          0.        ]
 [ 0.04317846  0.00462181  0.00996765  0.00639977]]
[[ 0.32020884  0.01009021  0.19225201  0.47744895]
 [ 0.21493002  0.78506998  0.          0.        ]
 [ 0.96159073  0.03840927  0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.67290026  0.07202708  0.15533756  0.0997351 ]]
[[1 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]]
